In [1]:
ACCESS_KEY = "*****************"
SECRET_KEY = "**************************************"
ENCODED_SECRET_KEY = SECRET_KEY.replace("/", "%2F")
AWS_BUCKET_NAME = "bigdatasp1"
MOUNT_NAME = "bd12may4"
dbutils.fs.mount("s3a://%s:%s@%s" % (ACCESS_KEY, ENCODED_SECRET_KEY, AWS_BUCKET_NAME), "/mnt/%s" % MOUNT_NAME)

In [2]:
display(dbutils.fs.ls("/mnt/%s" % MOUNT_NAME))

In [3]:
CV_data = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('dbfs:/mnt/bd12may4/Cleani3.csv')

In [4]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [5]:
spark = SparkSession.builder.appName('crime').getOrCreate()

In [6]:
indexer = StringIndexer(inputCol="Primary Type", outputCol="Primary_type")
indexed = indexer.fit(CV_data).transform(CV_data)
indexed.head(5)

In [7]:
indexer2 = StringIndexer(inputCol="Loc Description", outputCol="Loc_Description")
indexed1 = indexer2.fit(indexed).transform(indexed)
indexed1.head(5)

In [8]:
indexer2 = StringIndexer(inputCol="hours", outputCol="hours_cat")
indexed2 = indexer2.fit(indexed1).transform(indexed1)
indexed2.head(5)

In [9]:
assembler = VectorAssembler(
    inputCols=["Primary_type", "Loc_Description", "hours_cat", "District", "Domestic"],
    outputCol="features")

In [10]:
output = assembler.transform(indexed2)

In [11]:
train_output,test_output = output.randomSplit([0.7,0.3])

In [12]:
train_data = train_output.select("features", "Arrest")

In [13]:
lr1 = LogisticRegression(labelCol='Arrest', featuresCol="features", maxIter=10)

In [14]:
lrModel1 = lr1.fit(train_data)

In [15]:
test_data = test_output.select("features", "Arrest")

In [16]:
test_result = lrModel1.evaluate(test_data)

In [17]:
test_result2 = lrModel1.transform(test_data)

In [18]:
display(test_result2)

In [19]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="Arrest")
evaluator.evaluate(test_result2)

In [20]:
test_out_df = test_output.toPandas()

In [21]:
test_res_df = test_result2.toPandas()

In [22]:
test_out_df['predictions'] = test_res_df['prediction']

In [23]:
sd1 = sqlContext.createDataFrame(test_out_df)

In [24]:
sd1.show()

In [25]:
sd2 = sd1.drop('features')

In [26]:
sd1.repartition(1).write.format("csv").option("header", "true").save("s3a://bigdatasp1/results")

In [27]:
indexed2.registerTempTable("crime")

In [28]:
sqlContext.sql("select Primary_type from crime where district = 4").show()

In [29]:
sqlContext.sql("Select District, hours, COUNT(hours) from crime GROUP BY District, hours ORDER BY District, COUNT(hours)").show()